In [1]:
import glob
import zipfile
import json
import pandas as pd
import pickle
from pprint import pprint
import pandas as pd
from collections import defaultdict
import re
import scholarly

In [2]:
data_path = "/opt/datasets/microsoft_kdd/"

In [3]:
paper_file = data_path + "Papers.txt"

paper_dict = {}
title_dict = {}

errs = []

with open(paper_file, "r") as f:
    for line in f:
        try:
            paper_elems = line.split("\t")
            pid = paper_elems[0]
            year = int(paper_elems[3])
            normalized_title = ''.join(filter(str.isalpha, paper_elems[1]))
            if year == "2017":
                paper_dict[pid] = paper_elems[1:]
                title_dict[normalized_title] = pid
        except Exception as ex:
            errs.appendd(line)

In [5]:
with open("kdd_papers.pickle", "wb") as t:
    pickle.dump(paper_dict, t)

with open("kdd_title.pickle", "wb") as t:
    pickle.dump(title_dict, t)

In [6]:
len(errs)

0

In [8]:
len(title_dict)

0

## Read dblp citation data

In [13]:
ls ~/data/mag_aminer/dblp.v12.7z

/home/singh_shruti/data/mag_aminer/dblp.v12.7z


In [9]:
import py7zr

In [14]:
# Extracts the file in curent location unless path specified
with py7zr.SevenZipFile('/home/singh_shruti/data/mag_aminer/dblp.v12.7z', mode='r') as z:
    z.extractall()

In [22]:
cp dblp.v12.json /home/singh_shruti/data/mag_aminer/

In [1]:
import json

In [2]:
with open("dblp.v12.json", "r") as f:
    dblp_cit = json.load(f)

In [3]:
type(dblp_cit)

list

In [4]:
dblp_cit[0]

{'id': 1091,
 'authors': [{'name': 'Makoto Satoh',
   'org': 'Shinshu University',
   'id': 2312688602},
  {'name': 'Ryo Muramatsu', 'org': 'Shinshu University', 'id': 2482909946},
  {'name': 'Mizue Kayama', 'org': 'Shinshu University', 'id': 2128134587},
  {'name': 'Kazunori Itoh', 'org': 'Shinshu University', 'id': 2101782692},
  {'name': 'Masami Hashimoto', 'org': 'Shinshu University', 'id': 2114054191},
  {'name': 'Makoto Otani', 'org': 'Shinshu University', 'id': 1989208940},
  {'name': 'Michio Shimizu',
   'org': 'Nagano Prefectural College',
   'id': 2134989941},
  {'name': 'Masahiko Sugimoto',
   'org': 'Takushoku University, Hokkaido Junior College',
   'id': 2307479915}],
 'title': 'Preliminary Design of a Network Protocol Learning Tool Based on the Comprehension of High School Students: Design by an Empirical Study Using a Simple Mind Map',
 'year': 2013,
 'n_citation': 1,
 'page_start': '89',
 'page_end': '93',
 'doc_type': 'Conference',
 'publisher': 'Springer, Berlin, Hei

In [6]:
ls

dblp.v12.json                   mag_authors_pubs_cits_2017_iclr.pickle
Filter2016data.ipynb            mag_features_2017_iclr.pickle
kdd_papers.pickle               mag_papers.pickle
kdd_title.pickle                read_mag_data.ipynb
mag_augmented_2017_iclr.pickle  read_ms_kdd.ipynb


In [7]:
len(dblp_cit)

4894081

In [19]:
papers_2017 = {}

for inf in dblp_cit:
    if "year" in inf and inf["year"] and inf["year"] == 2017:
        try:
            lowercased = inf["title"].lower()
            key = ''.join(filter(str.isalpha, lowercased))
            papers_2017[key] = inf
        except Exception as ex:
            print(ex)

In [20]:
len(papers_2017)

308588

In [2]:
data_raw = pd.read_pickle('../features/data_features_csv-2017.pkl')
data = data_raw[["id", "title", "label", "authors"]]
org_papers = data.to_dict('index')

In [22]:
found = 0

for k,v in org_papers.items():
    t = v["title"].lower()
    t = ''.join(filter(str.isalpha, t))
    if t in papers_2017:
        org_papers[k]["found"] = True
        org_papers[k]["dblp_data"] = papers_2017[t]
        found += 1
    else:
        org_papers[k]["found"] = False

In [24]:
len(org_papers), found

(490, 386)

In [30]:
refs = 0

for k, v in org_papers.items():
    if v["found"] and "references" in v["dblp_data"] and v["dblp_data"]["references"]:        
        refs += len(v["dblp_data"]["references"])

In [31]:
refs #for 177

3207

In [32]:
3207/177

18.11864406779661

### Try external APIs

In [3]:
len(org_papers)

490

In [4]:
list(org_papers.keys())[0:5]

['HJWHIKqgl', 'SyK00v5xx', 'Hkg4TI9xl', 'Hk1iOLcle', 'H178hw9ex']

In [5]:
org_papers['HJWHIKqgl']

{'id': 'HJWHIKqgl',
 'title': 'Generative Models and Model Criticism via Optimized Maximum Mean Discrepancy',
 'label': 'Accept',
 'authors': ['Dougal J. Sutherland',
  'Hsiao-Yu Tung',
  'Heiko Strathmann',
  'Soumyajit De',
  'Aaditya Ramdas',
  'Alex Smola',
  'Arthur Gretton']}

In [6]:
all_authors = defaultdict(int)

count_auths = 0

for k,v in org_papers.items():
    if "authors" in v:
        count_auths += len(v["authors"])
        
        for a in v["authors"]:
            all_authors[a] += 1

In [7]:
count_auths

1809

In [8]:
len(all_authors)

1425

In [9]:
import scholarly

In [40]:
x = scholarly.search_author('Dougal Sutherland, Toyota Technological')

In [29]:
type(x)

generator

In [41]:
p = []
for i in x:
    p.append(i)

In [39]:
len(p)

0

In [43]:
p[0].citedby

729

In [46]:
sample_auth = p[0].fill()

In [49]:
len(sample_auth.publications)

36

In [62]:
print(p[0])

{'_filled': False,
 'affiliation': 'Professor of Vision Science, UC Berkeley',
 'citedby': 20174,
 'email': '@berkeley.edu',
 'id': 'Smr99uEAAAAJ',
 'interests': ['vision science', 'psychology', 'human factors', 'neuroscience'],
 'name': 'Martin Banks',
 'url_picture': 'https://scholar.google.com/citations?view_op=medium_photo&user=Smr99uEAAAAJ'}


In [54]:
p = re.compile(" [A-Z]\.? ")
p.sub(" ", "Dougal J Sutherland")

'Dougal Sutherland'

### Get info for all authors

In [ ]:
# author_info = {}

count = len(author_info)
p = re.compile(" [A-Z]\.? ")

for a in par1:
    if not a in author_info:
        if count % 10 == 0:
            print("Done out of {} is: {}".format(len(all_authors), count))
        auth_candidates = []
        try:
            c = scholarly.search_author(a)
            for i in c:
                auth_candidates.append((i, i.fill()))

            if not auth_candidates:
                # no author found
                a_clean = p.sub(" ", a)
                c = scholarly.search_author(a_clean)
                for i in c:
                    auth_candidates.append((i, i.fill()))

            author_info[a] = auth_candidates
        except Exception as ex:
            print("Move on")
        count += 1

Done out of 1425 is: 250
Done out of 1425 is: 260
Done out of 1425 is: 270
Done out of 1425 is: 280
Done out of 1425 is: 290
Done out of 1425 is: 300
Done out of 1425 is: 310
Done out of 1425 is: 320
Done out of 1425 is: 330
Done out of 1425 is: 340
Done out of 1425 is: 350
Done out of 1425 is: 360
Done out of 1425 is: 370
Done out of 1425 is: 380
Done out of 1425 is: 390
Done out of 1425 is: 400
Done out of 1425 is: 410
Done out of 1425 is: 420
Done out of 1425 is: 430
Done out of 1425 is: 440
Done out of 1425 is: 450
Done out of 1425 is: 460
Done out of 1425 is: 470
Done out of 1425 is: 480
Done out of 1425 is: 490
Done out of 1425 is: 500
Done out of 1425 is: 510
Done out of 1425 is: 520
Done out of 1425 is: 530


In [73]:
with open("author_info_scholar.pickle", "wb") as f:
    pickle.dump(author_info, f)

In [74]:
len(author_info)

700

In [75]:
len(list(all_authors.keys()))

1425

In [69]:
list_authors = list(all_authors.keys())
par1 = list_authors[0:700]
par2 = list_authors[700:]

In [81]:
par21 = list_authors[700:1100]
par22 = list_authors[1100:]
len(par21), len(par22)

(400, 325)

In [83]:
with open("split_authors_for_local.txt", "w") as f:
    f.write("\n".join(par22))

## For local only

In [9]:
%%bash
ls
unzip "author_info_scholar_compressed.zip"

author_info_scholar_compressed.zip
Filter2016data.ipynb
mag_augmented_2017_iclr.pickle
read_mag_data.ipynb
read_ms_kdd.ipynb
split_authors_for_local.txt
Archive:  author_info_scholar_compressed.zip
  inflating: author_info_scholar.pickle  


In [12]:
with open("author_info_scholar.pickle", "rb") as f:
    author_info = pickle.load(f)
print(len(author_info))

700


In [16]:
with open("split_authors_for_local.txt", "r") as f:
    par22 = [l.strip() for l in f]

len(par22), par22[2]

(325, 'Megan Leszczynski')

## Common 

In [37]:
# author_info = {}
len_all_authors = 1425 #len(all_authors)

count = len(author_info)
p = re.compile(" [A-Z]\.? ")

for a in par22:
    if not a in author_info:
        if count % 10 == 0:
            print("Done out of {} is: {}".format(len_all_authors, count))
        auth_candidates = []
        try:
            print(count, a)
            c = scholarly.search_author(a)
            for i in c:
                auth_candidates.append((i, i.fill()))

            if not auth_candidates:
                # no author found
                a_clean = p.sub(" ", a)
                c = scholarly.search_author(a_clean)
                for i in c:
                    auth_candidates.append((i, i.fill()))

            author_info[a] = auth_candidates
        except Exception as ex:
            print(ex)
            print("Move on")
        count += 1

729 Liu Yang
Done out of 1425 is: 730
730 Alex Gittens
731 Michael W. Mahoney
732 Hai Wang
733 Takeshi Onishi
734 David McAllester
735 Chang Xu
736 Gang Wang
737 Bailey Kong
738 Charless C. Fowlkes
739 Christopher Tensmeyer
Done out of 1425 is: 740
740 Tony Martinez
741 Édouard Grave
742 Moustapha Cissé
743 Hervé Jégou
744 Stefan Carlsson
745 Hossein Azizpour
746 Ali Razavian
747 Leslie N. Smith
748 Nicholay Topin
749 Haoqi Fan
Done out of 1425 is: 750
750 Kris M. Kitani
751 Nir Baram
752 Tom Zahavy
753 Shie Mannor
754 Kimin Lee
755 Jaehyung Kim
756 Song Chong
757 Jinwoo Shin
758 Hao Zhao
759 Ming Lu
Done out of 1425 is: 760
760 Li Zhang
761 Daniel Jiwoong Im
762 He Ma
763 Chris Dongjoo Kim
764 Graham Taylor
765 Jean Harb
766 Doina Precup
767 David Raposo
768 Adam Santoro
769 David Barrett
Done out of 1425 is: 770
770 Mehdi Mirza
771 Peter L. Bartlett
772 Matthieu Labeau
773 Alexandre Allauzen
774 Alexander G. Ororbia II
775 Dimitrios C. Gklezakos
776 Rajesh P. N. Rao
777 Yotam Hechtli

918 Charles Siegel
919 Stefano Teso
Done out of 1425 is: 920
920 Andrea Passerini
921 Abhishek Chaurasia
922 Sangpil Kim
923 Andrew Nystrom
924 John Hughes
925 David Harwath
926 James R. Glass
927 Parker Hill
928 Babak Zamirai
929 Shengshuo Lu
Done out of 1425 is: 930
930 Yu-Wei Chao
931 Michael Laurenzano
932 Mehrzad Samadi
933 Marios Papaefthymiou
934 Scott Mahlke
935 Thomas Wenisch
936 Jia Deng
937 Lingjia Tang
938 Jason Mars
939 Emily Denton
Done out of 1425 is: 940
940 Sam Gross
941 Rob Fergus
942 Thang D. Bui
943 Sujith Ravi
944 Vivek Ramavajjala
945 Eder Santana
946 Jose C Principe
947 Lingpeng Kong
948 Chris Alberti
949 Daniel Andor
Done out of 1425 is: 950
950 Ivan Bogatyy
951 David Weiss
952 Paris Smaragdis
953 Roy H. Campbell
954 Aryk Anderson
955 Kyle Shaffer
956 Artem Yankov
957 Court Corley
958 Nathan Hodas
959 Lei Shi
Done out of 1425 is: 960
960 Shikun Feng
961 Zhifan Zhu
962 Zhigang Yuan
963 Yuting Hu
964 Yongfeng Huang
965 Ben Krause
966 Iain Murray
967 Steve Renals
9

In [ ]:
with open("author_info_scholar.pickle", "wb") as f:
    pickle.dump(author_info, f)

## For local

In [39]:
with open("author_info_scholar_local.pickle", "wb") as f:
    pickle.dump(author_info, f)

In [38]:
count

1025

# Merge files

In [2]:
with open("author_info_scholar_local.pickle", "rb") as f:
    author_info = pickle.load(f)
len(author_info)

1007

In [3]:
%%bash
ls

author_info_scholar_compressed.zip
author_info_scholar_local_comp.zip
author_info_scholar_local.pickle
author_info_scholar.pickle
author_server_part.pickle
author_server_part.zip
Filter2016data.ipynb
mag_augmented_2017_iclr.pickle
read_mag_data.ipynb
read_ms_kdd.ipynb
split_authors_for_local.txt


In [6]:
ls

author_info_scholar_compressed.zip  Filter2016data.ipynb
author_info_scholar_local_comp.zip  mag_augmented_2017_iclr.pickle
author_info_scholar_local.pickle    read_mag_data.ipynb
author_info_scholar.pickle          read_ms_kdd.ipynb
author_server_part.pickle           split_authors_for_local.txt
author_server_part.zip


In [4]:
with open("author_server_part.pickle", "rb") as f:
    author_server_part = pickle.load(f)
len(author_server_part)

1097

In [5]:
list(author_server_part.keys())[0:10]

['',
 'Xiaolong Wang',
 'Krzysztof J. Geras',
 'Prabhat',
 'Yuxin Wu',
 'Sarath Chandar',
 'Marcin Moczulski',
 'Balázs Kégl',
 'Navdeep Jaitly',
 'Mikael Henaff']

In [6]:
author_server_part["Mikael Henaff"]

[(<scholarly.scholarly.Author at 0x7f898322c2b0>,
  <scholarly.scholarly.Author at 0x7f898322c2b0>)]

In [7]:
author_server_part["Mikael Henaff"][0]

(<scholarly.scholarly.Author at 0x7f898322c2b0>,
 <scholarly.scholarly.Author at 0x7f898322c2b0>)

In [8]:
author_server_part["Mikael Henaff"][0][0]

In [10]:
author_server_part["Mikael Henaff"][0][0].citedby

2521

In [11]:
author_server_part["Mikael Henaff"][0][0].citedby5y

2426

In [12]:
print(author_server_part["Mikael Henaff"][0][0])

{'_filled': True,
 'affiliation': 'Microsoft Research',
 'citedby': 2521,
 'citedby5y': 2426,
 'cites_per_year': {2012: 26,
                    2013: 19,
                    2014: 42,
                    2015: 113,
                    2016: 246,
                    2017: 404,
                    2018: 613,
                    2019: 798,
                    2020: 245},
 'coauthors': [<scholarly.scholarly.Author object at 0x7f898322c2e8>,
 'email': '@nyu.edu',
 'hindex': 13,
 'hindex5y': 13,
 'i10index': 14,
 'i10index5y': 14,
 'id': 'bX__wkYAAAAJ',
 'interests': ['Artificial Intelligence',
               'Deep Learning',
               'Reinforcement Learning'],
 'name': 'Mikael Henaff',
 'publications': [<scholarly.scholarly.Publication object at 0x7f898322c710>,
 'url_picture': 'https://scholar.google.com/citations?view_op=medium_photo&user=bX__wkYAAAAJ'}


In [30]:
print(author_server_part["Mikael Henaff"][0][1].publications[0])

{'_filled': False,
 'bib': {'title': 'The loss surfaces of multilayer networks', 'year': 2015},
 'citedby': 813,
 'id_citations': 'bX__wkYAAAAJ:zYLM7Y9cAGgC',
 'source': 'citations'}


In [26]:
totalCount = 0
c = 0

for a in author_server_part.values():
    for adisamb in a:
        totalCount += 1
        if "cites_per_year" in dir(adisamb[0]):
            c+=1
print(c)
print(totalCount)

8101
8101


In [31]:
# Merge authors files

all_authors_dict = author_server_part

for a in author_info:
    if not a in all_authors_dict:
        all_authors_dict[a] = author_info[a]

In [32]:
len(all_authors_dict)

1404

In [33]:
with open("authors_gscholar_scrapped.pickle", "wb") as f:
    pickle.dump(all_authors_dict, f)